In [3]:
import pandas as  pd
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
    
import matplotlib.pyplot as plt

In [30]:
# Loading data
df = pd.read_csv("C:\\Users\\gnirmal\\Documents\\Learning\\Basic Statistics\\difficile.csv")

In [31]:
df.head()

,person,dose,libido
0,1,1,3
1,2,1,2
2,3,1,1
3,4,1,1
4,5,1,4


In [32]:
#let’s drop the ‘person’ column since we don’t need it
df.drop('person', axis= 1, inplace= True)

# Recoding value from numeric to string
df['dose'].replace({1: 'placebo', 2: 'low', 3: 'high'}, inplace= True)
    
# Gettin summary statistics
rp.summary_cont(df['libido'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,libido,15.0,3.466667,1.76743,0.456349,2.487896,4.445437


In [7]:
#That’s good to see the data as a whole, but we are really interested in the data by dosing.
rp.summary_cont(df['libido'].groupby(df['dose']))

,N,Mean,SD,SE,95% Conf.,Interval
dose,,,,,,
high,5,5.0,1.581139,0.707107,3.450484,6.549516
low,5,3.2,1.303840,0.583095,1.922236,4.477764
placebo,5,2.2,1.303840,0.583095,0.922236,3.477764


In [8]:
#ANOVA with scipy.stats
#If using scipy.stats, the method needed is stats.f_oneway(). The general applied method looks like this:

#stats.f_oneway(data_group1, data_group2, data_group3, data_groupN)
stats.f_oneway(df['libido'][df['dose'] == 'high'], 
             df['libido'][df['dose'] == 'low'],
             df['libido'][df['dose'] == 'placebo'])

F_onewayResult(statistic=5.11864406779661, pvalue=0.024694289538222603)

In [ ]:
#The F-statistic= 5.119 and the p-value= 0.025 which is indicating that there is an overall significant effect of medication 
#on libido. However, we don’t know where the difference between dosing/groups is yet. This is in the post-hoc section

In [ ]:
#ANOVA with statsmodels
#Using statsmodels, we get a bit more information and enter the model as a regression formula. 
#The general input using this method looks like this:

#model_name = ols('outcome_variable ~ group1 + group2 + groupN', data=your_data).fit()

In [9]:
results = ols('libido ~ C(dose)', data=df).fit()
results.summary()

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:1416: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 libido   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     5.119
Date:                Fri, 28 Feb 2020   Prob (F-statistic):             0.0247
Time:                        10:56:04   Log-Likelihood:                -24.683
No. Observations:                  15   AIC:                             55.37
Df Residuals:                      12   BIC:                             57.49
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              5.0000      0.627      7.972      0.000       3.634       6.366
C(dose)[T.low]        -1.8000      0.887     -2.029      0.065      -3.732       0.132
C(dose)[T.placebo]    -2.8000      0.887     -3.157      0.008      -4.732      -0.868
==============================================================================
Omnibus:                        2.517   Durbin-Watson:                   2.408
Prob(Omnibus):                  0.284   Jarque-Bera (JB):                1.108
Skew:                           0.195   Prob(JB):                        0.575
Kurtosis:                       1.727   Cond. No.                         3.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
aov_table = sm.stats.anova_lm(results, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(dose),20.133333,2.0,5.118644,0.024694
Residual,23.600000,12.0,NaN,NaN


In [11]:
#The following function calculates the effect sizes, as well as the mean squares and updates the table
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(aov_table)

,sum_sq,df,mean_sq,F,PR(>F),eta_sq,omega_sq
C(dose),20.133333,2.0,10.066667,5.118644,0.024694,0.460366,0.354486
Residual,23.600000,12.0,1.966667,NaN,NaN,NaN,NaN


#1-Way ANOVA by hand (from scratch)

In [34]:
data=df
data.head()

,dose,libido
0,placebo,3
1,placebo,2
2,placebo,1
3,placebo,1
4,placebo,4


In [35]:
# compute overall mean
overall_mean = data['libido'].mean()
overall_mean

3.466666666666667

In [36]:
# compute Sum of Squares Total
data['overall_mean'] = overall_mean
ss_total = sum((data['libido'] - data['overall_mean'])**2)
ss_total

43.73333333333333

In [37]:
# compute group means
group_means = data.groupby('dose').mean()
group_means = group_means.rename(columns = {'libido': 'group_mean'})
group_means

,group_mean,overall_mean
dose,,
high,5.0,3.466667
low,3.2,3.466667
placebo,2.2,3.466667


In [38]:
# add group means and overall mean to the original data frame
data = data.merge(group_means, left_on = 'dose', right_index = True)

In [39]:
# compute Sum of Squares Residual
ss_residual = sum((data['libido'] - data['group_mean'])**2)
ss_residual

23.6

In [41]:
data.head()

,dose,libido,overall_mean_x,group_mean,overall_mean_y
0,placebo,3,3.466667,2.2,3.466667
1,placebo,2,3.466667,2.2,3.466667
2,placebo,1,3.466667,2.2,3.466667
3,placebo,1,3.466667,2.2,3.466667
4,placebo,4,3.466667,2.2,3.466667


In [42]:
# compute Sum of Squares Model
ss_explained = sum((data['overall_mean_x'] - data['group_mean'])**2)
ss_explained

20.133333333333333

In [43]:
# compute Mean Square Residual
n_groups = len(set(data['dose']))
n_obs = data.shape[0]
df_residual = n_obs - n_groups
ms_residual = ss_residual / df_residual
ms_residual

1.9666666666666668

In [44]:
# compute Mean Square Explained
df_explained = n_groups - 1
ms_explained = ss_explained / df_explained
ms_explained

10.066666666666666

In [45]:
# compute F-Value
f = ms_explained / ms_residual
f

5.11864406779661

In [46]:
# compute p-value
import scipy.stats
p_value = 1 - scipy.stats.f.cdf(f, df_explained, df_residual)
p_value

0.024694289538222614

#Post-hoc Tests